# 计算用户和app的匹配程度（只计算与正例的接近程度）

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
# 读入用户和app
dataTrain = pd.concat([pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv"),pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\test.csv")])
dataTrain  = pd.merge(dataTrain, pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv"), on='creativeID')
dataTrain = dataTrain[['userID','appID']]
## 去掉重复值
dataTrain.drop_duplicates(inplace=True)

In [3]:
# 读入userSingle
## 注意这里使用了single2，即app出现数量log化了，更鼓励小众app
dataUserSingle = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_single_simple4.csv')
dictUserSingle = {}
def getDictUserSingle(x):
    key = str(int(x[0])) + '-' + str(int(x[1])) + '-' + str(int(x[2]))
    value = eval(x[3])
    dictUserSingle[key] = value
dataUserSingle.apply(getDictUserSingle, axis=1)
print dictUserSingle['83-1-0']['723'] #第一位是appid，第二位是single，第三位是partition

0.0113160574856


In [4]:
dataAppCoScale = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\app_coScale.csv')
dataAppCoScale['scale_reci'] = 1000000/dataAppCoScale['scale'] #扩大系数使得不用省略太多尾数
dataAppCoScale =  dataAppCoScale[['appID', 'scale_reci']].set_index('appID')
dictAppScore = dataAppCoScale.to_dict() #索引，为了加快速度

In [5]:
# 读取user_apps
dataUserApps = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_apps.csv', names=['userID', 'appList'])
dict_userApps = {}
def getDictUserApps(x):
     dict_userApps[x[0]] = x[1][1:-1].split(', ')
dataUserApps.apply(getDictUserApps, axis=1)
dataUserApps.head()

,userID,appList
0,1,"[357, 360, 362, 365, 375, 441, 480, 481, 482, ..."
1,9,"[538, 602, 604]"
2,10,"[357, 359, 365, 383, 480, 481, 482, 483, 484, ..."
3,12,"[480, 481, 482, 483, 484, 485, 486, 487, 488, ..."
4,14,"[362, 480, 481, 482, 483, 484, 485, 486, 487, ..."


In [6]:
# 直接计算好single-appid的score，加快运算速度
dict_SingleAppid_score = {}
# 获得targetID名单
dataAppID = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv')
appIDs = set(dataAppID['appID'])
for id in appIDs:
    voteType = [0, 1]
    for v in voteType:
        single = str(id) + '-' + '1' + '-' + str(v)
        mDict = dictUserSingle[single]
        for appid in mDict.keys():
            dict_SingleAppid_score[single + '|'  + str(appid)] = dictAppScore['scale_reci'][int(appid)] * mDict[appid]

In [7]:
# 每个single的keys
dict_single_keys = {}
appIDs = set(dataAppID['appID'])
for id in appIDs:
    voteType = [0, 1]
    for v in voteType:
        single = str(id) + '-' + '1' + '-' + str(v)
        dict_single_keys[single] = dictUserSingle[single].keys()

In [8]:
# 用户与single的相似度计算函数
def getScore(list, single):
    score = 0.0
    for l in list:
#         无并集模式
#         score += dict_SingleAppid_score[single + '|' + l]
#         ## 有并集模式
        if dictUserSingle[single].has_key(l): #交集及single这边的app
            score += dict_SingleAppid_score[single + '|' + l]
        else: #仅user这边的app
            score += dictAppScore['scale_reci'][int(l)]
    return score
def cal_coScale(user, single):
    if dict_userApps.has_key(user) == False: #对于缺乏数据的用户，目前返回0
        return 0
    list0 = dict_userApps[user]
    list1 = dict_single_keys[single]
#     intersect = np.intersect1d(list0, list1)
    union = np.union1d(list0, list1)
#     length = len(intersect)
#     if(length == 0): 
#         return 0
#     else: 
#         return getScore(intersect, single)
#         return getScore(intersect, single)/length
#         return getScore(intersect, single)/getScore(union, single)
    return getScore(union, single)

In [9]:
def getVote(x, voteType):
    single = str(int(x[1])) + '-' + '1' + '-' + str(voteType)
    return cal_coScale(int(x[0]), single)
dataTrain['vote1_union'] = dataTrain[['userID','appID']].apply(getVote, args=(1,), axis=1)
# dataTrain['vote0'] = dataTrain[['userID','appID']].apply(getVote, args=(0,), axis=1) #不计算0了

In [ ]:
import timeit
def ft():
    cal_coScale(16,'465-1-1')
timeit.timeit(ft,number=1)

In [ ]:
dataTrain.head()

In [ ]:
dataTrain['voteMinus1'] = dataTrain['vote1'] - dataTrain['vote0']
dataTrain['voteMinus2'] = (dataTrain['vote1']-dataTrain['vote0'])/(dataTrain['vote1']+dataTrain['vote0']+1)

In [ ]:
dataTrain[dataTrain['voteMinus1']==0].groupby(['label']).count()

In [10]:
# 4是采用simple的方法获取single，并采用 getScore(intersect, single)/length 获取vote1和vote0
# 5是采用有threshold的simple方法(阈值千分一)
# simple3是采用有threshold的simple方法(阈值万分一)
# simple4是无log化，阈值为万分五的方法,用getscore/length的方法求vote1
# simple4_是求simple的getScore(union,single)
dataTrain.to_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\user_single_vote_simple4_.csv',index=False)

# 进一步处理

In [15]:
temp = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\user_single_vote_simple4.csv')
temp = temp.drop_duplicates()
temp2 = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\user_single_vote_simple4_.csv')
temp2 = temp2.drop_duplicates()
temp = pd.merge(temp, temp2, on=['userID','appID'])
temp['vote1++'] = temp['vote1'] / temp['vote1_union']

In [22]:
dataTrain = pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv")[['userID','label']]
temp = pd.merge(dataTrain, temp, on='userID', how='left')

In [24]:
temp[(temp['appID']==83)].sort_values(['vote1'],ascending=False)

,userID,label,appID,vote1,vote1_union,vote1++
2782372,2175688,1,83,2.006151,1.125801e+06,1.781976e-06
144689,1516447,1,83,1.388120,8.380563e+02,1.656356e-03
2068540,1396737,1,83,1.363582,5.057466e+03,2.696177e-04
950696,1002040,0,83,1.172221,1.984023e+06,5.908303e-07
2349247,1540114,1,83,1.167720,1.127241e+03,1.035910e-03
878598,1315426,0,83,1.081814,1.180070e+05,9.167372e-06
765835,2737714,1,83,0.967201,1.542548e+05,6.270153e-06
462403,896582,1,83,0.966480,1.464864e+06,6.597750e-07
544514,1964691,1,83,0.901216,1.210712e+05,7.443686e-06
1403129,914826,0,83,0.871102,8.380563e+02,1.039432e-03


In [ ]:
temp['voteMinus'] = temp['vote1'] - temp['vote0']
# temp['voteMinus'] = (temp['vote1']-temp['vote0'])/(temp['vote1'] + temp['vote0']+1)

In [ ]:
dfAppidVotemax = temp[['appID','voteMinus']].groupby('appID').max()
dfAppidVotemean = temp[['appID','voteMinus']].groupby('appID').mean()
dfAppidVotemin = temp[['appID','voteMinus']].groupby('appID').min()

In [ ]:
dfAppidVotemax.at[83,'voteMinus']

In [ ]:
def minusNor(x):
    appid = int(x[0])
    _max = dfAppidVotemax.at[appid, 'voteMinus']
    _min = dfAppidVotemin.at[appid, 'voteMinus']
    return (x[1]-_min)/(_max-_min)
temp['voteMinus_nor'] = temp[['appID','voteMinus']].apply(minusNor, axis=1)

## 为每个appID单独生成字段(user_single_vote2)

In [ ]:
# 获得targetID名单
dataAppID = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv')
appIDs = set(dataAppID['appID'])
# 为每个appID单独生成字段
for id in appIDs:
    temp['vm' + str(id) + '_1'] = (temp['appID']==id).astype(int) * temp['voteMinus']
    temp['vm' + str(id) + '_0'] = (temp['voteMinus']==0).astype(int)

In [ ]:
temp[temp['appID']==83][['appID','vm83_1','vm83_0']].head()

In [ ]:
out = temp.drop(['label','vote1','vote0','voteMinus'],axis=1)

In [ ]:
out.to_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\user_single_vote2.csv',index=False)

## 其它数据分析

In [ ]:
tempTrain = pd.concat([pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv"),pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\test.csv")])
tempTrain  = pd.merge(tempTrain, pd.read_csv("C:\\Users\\work\\Desktop\\tencent\\pre\\ad.csv"), on='creativeID')

In [ ]:
dataUserAppscale = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\app-user\\user_appScale.csv')

In [ ]:
temp = pd.merge(temp, dataUserAppscale, on='userID')

In [ ]:
tempList1 = dict_userApps[2547462]
tempList2 = dictUserSingle['150-1-1'].keys()
# print np.intersect1d(tempList1, tempList2)
# print np.union1d(tempList1, tempList2)
print dict_userApps[2547462]
print dict_userApps[2547462]

In [ ]:
tempTrain[tempTrain['userID']==2517914][['label','userID','appID','clickTime','adID']].sort_values(by=['clickTime'],ascending=True)

In [ ]:
temp.groupby('appID').count().sort_values(['label'],ascending=False)

In [ ]:
# temp[(temp['appID']==83)&(temp['voteMinus']>0)].groupby('label').count()
# temp.sort_values(by=['voteMinus2'],ascending=False)[:50] #整体排序
temp[(temp['voteMinus1']>0)].sort_values(by=['voteMinus1'],ascending=False)[:50] #各APP正例排序
# temp[(temp['appID']==428)&(temp['voteMinus']<0)].sort_values(by=['voteMinus'],ascending=True)[:50] #各APP负例排序

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import  MultipleLocator
temp2 = temp[(temp['label']!=-1)&(temp['voteMinus']!=0)&(temp['appID']==465)].sort_values(by=['voteMinus'],ascending=False)
roll = temp2['label'].rolling(50, min_periods=1).mean()
dataR = temp2['scale'].rolling(50, min_periods=1).mean()
axL = plt.subplot(111)
axL.plot(range(len(roll)), roll.values, color='blue')
axR = axL.twinx()
axR.plot(range(len(roll)), dataR.values, color='red')
# xminorLocator = MultipleLocator(1)
# axL.xaxis.set_major_locator(xminorLocator)
plt.show()